In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from functools import reduce # needed for merging multiple dataframes

In [10]:
mean_sale_price = pd.read_csv(r'datasets/Mean Sales Price of Houses Sold.csv')
median_sale_price = pd.read_csv(r'datasets/Median Sales Price of Houses Sold.csv')
house_price_index = pd.read_csv(r'datasets/All-Transactions House Price Index.csv')
median_income = pd.read_csv(r'datasets/Median Household Income.csv')

print("Mean sales price: \n" + str(mean_sale_price.head()))
print(str(mean_sale_price.info()) + "\n\n")
print("Median sales price: \n" + str(median_sale_price.head()))
print(str(median_sale_price.info()) + "\n\n")
print("House price index: \n" + str(house_price_index.head()))
print(str(house_price_index.info()) + "\n\n")
print("Median household income: \n" + str(median_income.head()))
print(str(median_income.info()) + "\n\n")

Mean sales price: 
  observation_date  ASPUS
0       1963-01-01  19300
1       1963-04-01  19400
2       1963-07-01  19200
3       1963-10-01  19600
4       1964-01-01  19600
<class 'pandas.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   observation_date  252 non-null    str  
 1   ASPUS             252 non-null    int64
dtypes: int64(1), str(1)
memory usage: 4.1 KB
None


Median sales price: 
  observation_date  MSPUS
0       1963-01-01  17800
1       1963-04-01  18000
2       1963-07-01  17900
3       1963-10-01  18500
4       1964-01-01  18500
<class 'pandas.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   observation_date  252 non-null    str  
 1   MSPUS             252 non-null    int64
dtypes: int64(1), str(1)
memory usage: 4.1 KB
Non

mean sales price, median sales price, and house price index are all quarterly data, while median household income is yearly data.

housing datasets start from 193 (mean and median sales price) and 1953 (house price index), while median household income starts from 1967.

The 'observation_date' column is a string instead of datetime format for all datasets.

In [16]:
# change 'observation_date' column to datetime format for all datasets - needed to make quaterly -> yearly data, mergeing datasets (based on date), and for plotting time series data
mean_sale_price['observation_date'] = pd.to_datetime(mean_sale_price['observation_date'])
median_sale_price['observation_date'] = pd.to_datetime(median_sale_price['observation_date'])
house_price_index['observation_date'] = pd.to_datetime(house_price_index['observation_date'])
median_income['observation_date'] = pd.to_datetime(median_income['observation_date'])
mean_sale_price.info()

<class 'pandas.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   observation_date  252 non-null    datetime64[us]
 1   ASPUS             252 non-null    int64         
dtypes: datetime64[us](1), int64(1)
memory usage: 4.1 KB


In [20]:
# creates 'Year' column for all datasets by extrating year from 'observation_date' column
mean_sale_price['Year'] = mean_sale_price['observation_date'].dt.year
median_sale_price['Year'] = median_sale_price['observation_date'].dt.year
house_price_index['Year'] = house_price_index['observation_date'].dt.year
median_income['Year'] = median_income['observation_date'].dt.year
mean_sale_price.head()


,observation_date,ASPUS,Year
0,1963-01-01,19300,1963
1,1963-04-01,19400,1963
2,1963-07-01,19200,1963
3,1963-10-01,19600,1963
4,1964-01-01,19600,1964


In [29]:
# creates (new) yearly dataframes by grouping by 'Year' column and taking mean of sales price, house price index, and median household income for each year
mean_yearly = mean_sale_price.groupby('Year')['ASPUS'].mean().reset_index()
median_yearly = median_sale_price.groupby('Year')['MSPUS'].mean().reset_index()
hpi_yearly = house_price_index.groupby('Year')['USSTHPI_NBD19750101'].mean().reset_index()
income_yearly = median_income.groupby('Year')['MEHOINUSA646N'].mean().reset_index()
mean_yearly.head()

,Year,ASPUS
0,1963,19375.0
1,1964,20300.0
2,1965,21450.0
3,1966,22925.0
4,1967,24125.0


In [ ]:
# pd.merge can combine only two DataFrames at a time.
# lambda left, right: defines one merge step for a pair of DataFrames.
# It merges on 'Year' using an inner join.
# reduce(...) applies that same merge step repeatedly across the list.
# Process: merge df1+df2, then merge result with df3, then with df4, etc.
# Result: one final DataFrame containing only Years common to all inputs.

dataframes_list = [mean_yearly, median_yearly, hpi_yearly, income_yearly]

merged = reduce(lambda left, right: pd.merge(left, right, on='Year', how='inner'), dataframes_list)

merged.head()

,Year,ASPUS,MSPUS,USSTHPI_NBD19750101,MEHOINUSA646N
0,1984,97550.0,79950.0,202.221205,22420.0
1,1985,100825.0,84275.0,212.547925,23620.0
2,1986,112075.0,92025.0,227.071178,24900.0
3,1987,127575.0,104700.0,241.765295,26060.0
4,1988,138650.0,112225.0,254.717452,27230.0
